In [ ]:
import csv
import json
import pandas as pd

In [16]:
# Charger le fichier CSV
df = pd.read_csv("interlangue_automatique_corrigé.csv", sep=",")

# Charger les alignements manuel/correct
with open("alignement_mots_manuel_correct.json", encoding="utf-8") as f:
    alignement_data = json.load(f)

# Charger les transcriptions ordonnées
with open("transcriptions_ordonnées.json", encoding="utf-8") as f:
    transcriptions_data = json.load(f)

# Créer un dictionnaire d'alignements {(pays, apprenant): {mot_interlangue: [mot_correct1, mot_correct2, ...]}}
alignement_dict = {}
for entry in alignement_data:
    key = (entry["pays"], str(entry["apprenant"]))
    if key not in alignement_dict:
        alignement_dict[key] = {}
    for segment in entry["segments"]:
        for pair in segment["segments"]:
            manuel = pair["manuel"]
            correct = pair["correct"]
            if manuel not in alignement_dict[key]:
                alignement_dict[key][manuel] = []
            alignement_dict[key][manuel].append(correct)

# Créer un dictionnaire {(pays, apprenant): version complète correcte en polonais}
version_correcte_dict = {}
for entry in transcriptions_data:
    key = (entry["pays"], str(entry["numéro d'apprenant"]))
    phrases = []
    i = 1
    while f"version correcte en polonais ({i})" in entry:
        phrase = entry[f"version correcte en polonais ({i})"]
        phrases.append(phrase.strip())
        i += 1
    version_correcte_dict[key] = " ".join(phrases)

# Générer la colonne "mot (correct en polonais)" avec correspondance unique
# Pour chaque (pays, apprenant, mot_interlangue), suivre un index d'utilisation
usage_index = {}
correct_words = []

for idx, row in df.iterrows():
    pays = row["pays"]
    apprenant = str(row["apprenant"])
    mot_inter = row["mot (interlangue)"]
    key = (pays, apprenant)
    mot_correct = ""

    if key in alignement_dict and mot_inter in alignement_dict[key]:
        liste_possibles = alignement_dict[key][mot_inter]
        usage_key = (pays, apprenant, mot_inter)
        index = usage_index.get(usage_key, 0)

        if index < len(liste_possibles):
            mot_correct = liste_possibles[index]
            usage_index[usage_key] = index + 1  # augmenter l'index pour la prochaine occurrence

    correct_words.append(mot_correct)

# Insérer la colonne juste après "mot (automatique)"
df.insert(df.columns.get_loc("mot (automatique)") + 1, "mot (correct en polonais)", correct_words)

# Générer la colonne "version correcte en polonais"
version_correcte_col = []
déjà_ajoutés = set()

for idx, row in df.iterrows():
    key = (row["pays"], str(row["apprenant"]))
    if key in version_correcte_dict and key not in déjà_ajoutés:
        version_correcte_col.append(version_correcte_dict[key])
        déjà_ajoutés.add(key)
    else:
        version_correcte_col.append("")

# Insérer la colonne après "mot (correct en polonais)"
df.insert(df.columns.get_loc("mot (correct en polonais)") + 1, "version correcte en polonais", version_correcte_col)

# Enregistrer le fichier CSV final
df.to_csv("interlangue_automatique_correct.csv", sep=",", index=False, encoding="utf-8")

# Afficher des statistiques sur les alignements réussis (hors "—")
total = len(df)
alignés = sum(1 for mot in correct_words if mot.strip() != "" and mot.strip() != "—")
pourcentage = (alignés / total) * 100 if total > 0 else 0

print(f"Nombre total d'exemples : {total}")
print(f"Nombre d'exemples avec mot correct trouvé (hors '—') : {alignés}")
print(f"Pourcentage d'alignements réussis : {pourcentage:.2f}%")


Nombre total d'exemples : 686
Nombre d'exemples avec mot correct trouvé (hors '—') : 612
Pourcentage d'alignements réussis : 89.21%


##### Correction du fichier `interlangue_automatique_correct.csv` -> `interlangue_automatique_correct_corrigé.csv`

In [2]:
# Lire le fichier CSV d'entrée
df = pd.read_csv("interlangue_automatique_correct_corrigé.csv", encoding="utf-8")

# Supprimer les lignes où la colonne "erreur de prononciation?" est vide ou NaN
df = df[df["erreur de prononciation?"].notna() & (df["erreur de prononciation?"] != "")]

# Supprimer les colonnes "erreur de déclinaison?" et "erreur de prononciation?"
df = df.drop(columns=["erreur de déclinaison?", "erreur de prononciation?"])

# Sauvegarder le fichier filtré dans un nouveau fichier CSV
df.to_csv("interlangue_prononciation.csv", index=False, encoding="utf-8")
